In [1]:
from IPython.core.getipython import get_ipython

get_ipython().run_line_magic("load_ext", "autoreload")
get_ipython().run_line_magic("autoreload", "2")

from datetime import datetime, timedelta
from pathlib import Path
import logging
from hydra import compose, initialize

import numpy as np
from omegaconf import DictConfig
from sous_chef.date.get_due_date import DueDatetimeFormatter
# from sous_chef.formatter.format_unit import UnitFormatter
# from sous_chef.formatter.ingredient.format_ingredient import IngredientFormatter
from sous_chef.menu.create_menu.create_menu import Menu
from sous_chef.menu.create_menu._from_fixed_template import FixedTemplates
from sous_chef.menu.record_menu_history import MenuHistorian
# from sous_chef.pantry_list.read_pantry_list import PantryList
from sous_chef.recipe_book.read_recipe_book import RecipeBook
from sous_chef.rtk.read_write_rtk import RtkService
from sous_chef.menu.main import _get_ingredient_formatter
from structlog import get_logger

from utilities.api.gsheets_api import GsheetsHelper
# from utilities.api.todoist_api import TodoistHelper

In [2]:
def update_data():
    with initialize(
            version_base=None, config_path="../../sous-chef/config/"
    ):
        config = compose(config_name="menu_main")
        rtk_service = RtkService(config.rtk)
        rtk_service.unzip()

def get_recipe_book():
    with initialize(
            version_base=None, config_path="../../sous-chef/config/"
    ):
        config = compose(config_name="menu_main")
        return RecipeBook(config.recipe_book)
    
def get_menu_historian():
    with initialize(
            version_base=None, config_path="../../sous-chef/config/"
    ):
        config = compose(config_name="menu_main")
        gsheets_helper = GsheetsHelper(config.api.gsheets)
        due_date_formatter = DueDatetimeFormatter(config=config.date.due_date)

        menu_historian = MenuHistorian(
            config=config.menu.record_menu_history,
            current_menu_start_date=due_date_formatter.get_anchor_datetime()
                                    + timedelta(days=1),
            gsheets_helper=gsheets_helper,
        )
        
        return menu_historian
    
def create_menu(recipe_book: RecipeBook, menu_historian: MenuHistorian) -> Menu:
    with initialize(
            version_base=None, config_path="../../sous-chef/config/"
    ):
        config = compose(config_name="menu_main")
        
        gsheets_helper = GsheetsHelper(config.api.gsheets)
        due_date_formatter = DueDatetimeFormatter(config=config.date.due_date)
        ingredient_formatter = _get_ingredient_formatter(config, gsheets_helper)

        menu = Menu(
            config=config.menu.create_menu,
            due_date_formatter=due_date_formatter,
            gsheets_helper=gsheets_helper,
            ingredient_formatter=ingredient_formatter,
            menu_historian=menu_historian,
            recipe_book=recipe_book,
        )
    return menu

def get_menu_history_uuids(menu_historian: MenuHistorian):
    with initialize(
            version_base=None, config_path="../../sous-chef/config/"
    ):
        config = compose(config_name="menu_main")
        
        history = menu_historian.get_history_from(
            days_ago=config.menu.create_menu.menu_history_recent_days
        )
        
        return list(history.uuid.values)
    
def get_future_menu_uuids(menu: Menu, future_menus: int = None):

        fixed_templates = FixedTemplates(
            config=menu.config.fixed,
            due_date_formatter=menu.due_date_formatter,
            gsheets_helper=menu.gsheets_helper,
        )

        future_uuid_tuple = menu._get_future_menu_uuids(
            future_menus=fixed_templates.select_upcoming_menus(
                num_weeks_in_future=future_menus if future_menus else menu.config.fixed.already_in_future_menus.num_weeks  # noqa: E501
            )
        )
        
        return future_uuid_tuple
    

In [3]:
# logging.getLogger().setLevel(logging.CRITICAL)

update_data()
recipes = get_recipe_book()

2024-03-08 15:29:43 [info     ] [format recipe row]            recipe= Lentil Soup
2024-03-08 15:29:43 [info     ] [format recipe row]            recipe=15-Hour TikTok Potatoes
2024-03-08 15:29:43 [info     ] [format recipe row]            recipe=2-Layer Lemon Bites
2024-03-08 15:29:43 [info     ] [format recipe row]            recipe=2-Minute Healthy Ketchup
2024-03-08 15:29:43 [warning  ] [create_timedelta] conversion failed entry=
2024-03-08 15:29:43 [warning  ] [create_timedelta] conversion failed entry=
2024-03-08 15:29:43 [info     ] [format recipe row]            recipe=3 Bean Stew with Rice & Vegetables
2024-03-08 15:29:43 [warning  ] [create_timedelta] conversion failed entry=
2024-03-08 15:29:43 [info     ] [format recipe row]            recipe=3 bean salad
2024-03-08 15:29:43 [info     ] [format recipe row]            recipe=3-Ingredient Orange Chicken
2024-03-08 15:29:43 [warning  ] [create_timedelta] conversion failed entry=
2024-03-08 15:29:43 [info     ] [format recipe r

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



2024-03-08 15:29:46 [info     ] [format recipe row]            recipe=Foolproof Pan Pizza Recipe
2024-03-08 15:29:46 [warning  ] [create_timedelta] conversion failed entry=
2024-03-08 15:29:46 [warning  ] [create_timedelta] conversion failed entry=
2024-03-08 15:29:46 [warning  ] [create_timedelta] conversion failed entry=
2024-03-08 15:29:46 [info     ] [format recipe row]            recipe=Forbidden Forrest Butterbeer Chocolate Cake
2024-03-08 15:29:46 [warning  ] [create_timedelta] conversion failed entry=
2024-03-08 15:29:46 [warning  ] [create_timedelta] conversion failed entry=
2024-03-08 15:29:46 [warning  ] [create_timedelta] conversion failed entry=
2024-03-08 15:29:46 [warning  ] [create_timedelta] conversion failed entry=
2024-03-08 15:29:46 [info     ] [format recipe row]            recipe=Fragrant Spicy Lotus Root | 香辣扮莲藕
2024-03-08 15:29:46 [warning  ] [create_timedelta] conversion failed entry=
2024-03-08 15:29:46 [info     ] [format recipe row]            recipe=Freckle

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



2024-03-08 15:29:50 [warning  ] [create_timedelta] conversion failed entry=
2024-03-08 15:29:50 [info     ] [format recipe row]            recipe=Chocolate Truffle Pie
2024-03-08 15:29:50 [warning  ] [create_timedelta] conversion failed entry=
2024-03-08 15:29:50 [warning  ] [create_timedelta] conversion failed entry=
2024-03-08 15:29:50 [info     ] [format recipe row]            recipe=Chocolate Yogurt Loaf
2024-03-08 15:29:50 [warning  ] [create_timedelta] conversion failed entry=
2024-03-08 15:29:50 [warning  ] [create_timedelta] conversion failed entry=
2024-03-08 15:29:50 [info     ] [format recipe row]            recipe=Chocolate Yogurt Parfaits with Raspberries
2024-03-08 15:29:50 [warning  ] [create_timedelta] conversion failed entry=
2024-03-08 15:29:50 [warning  ] [create_timedelta] conversion failed entry=
2024-03-08 15:29:50 [info     ] [format recipe row]            recipe=Chocolate Zucchini Banana Muffins
2024-03-08 15:29:50 [warning  ] [create_timedelta] conversion faile

In [4]:
menu_historian = get_menu_historian()
menu = create_menu(recipes, menu_historian)

2024-03-08 15:29:51 [info     ] [get_worksheet]                workbook_name=menu-history worksheet_name=menu-history
2024-03-08 15:29:55 [info     ] [get_worksheet]                workbook_name=pantry_list worksheet_name=ingredients
2024-03-08 15:29:57 [info     ] [get_worksheet]                workbook_name=pantry_list worksheet_name=replacement_ingredients
2024-03-08 15:29:58 [info     ] [get_worksheet]                workbook_name=pantry_list worksheet_name=bad_ingredients
2024-03-08 15:29:59 [info     ] [get_worksheet]                workbook_name=pantry_list worksheet_name=misspelled_ingredients
2024-03-08 15:30:01 [warning  ] Empty DataFrame
Columns: [replacement_ingredient, plural_ending, replace_factor, replace_unit, true_ingredient, item_plural, misspelled_ingredient, label, _merge]
Index: []
2024-03-08 15:30:01 [warning  ] Empty DataFrame
Columns: [ingredient, plural_ending, is_staple, group, store, recipe_uuid, barcode, item_plural, misspelled_ingredient, true_ingredient, l

In [5]:
current_menu = menu.load_final_menu()
menu.config.fixed.menu_number = 6

menu_history = get_menu_history_uuids(menu_historian)
menu_future = get_future_menu_uuids(menu, future_menus=15)

2024-03-08 15:30:01 [info     ] [get_worksheet]                workbook_name=menu-tmp worksheet_name=menu-tmp
2024-03-08 15:30:03 [info     ] [_get_all_fixed_menus]
2024-03-08 15:30:03 [info     ] [get_worksheet]                workbook_name=fixed_menus worksheet_name=breakfast
2024-03-08 15:30:06 [info     ] [get_worksheet]                workbook_name=fixed_menus worksheet_name=snack
2024-03-08 15:30:09 [info     ] [get_worksheet]                workbook_name=fixed_menus worksheet_name=dinner
2024-03-08 15:30:13 [info     ] [get_worksheet]                workbook_name=fixed_menus worksheet_name=dessert
2024-03-08 15:30:15 [info     ] [select_upcoming_menus]
2024-03-08 15:30:15 [info     ] [_get_future_menu_uuids]


RecipeNotFoundError: [recipe not found] recipe=joint-snack/special search_results=[[fuzzy search failed]: {'field': 'title', 'search_term': 'joint-snack/special', 'result': 'lentil spinach salad', 'quality': 56, 'threshold': 90}]

In [ ]:
recipe_df = recipes.dataframe.copy()
recipe_df.reset_index(drop=True)
display(recipe_df)

In [ ]:
old_len = recipe_df.shape[0]
recipe_history_df = recipe_df[recipe_df.uuid.isin(menu_history)]

recipe_df = recipe_df[~recipe_df.uuid.isin(menu_history)]
recipe_df = recipe_df[~recipe_df.uuid.isin(menu_future)]
print(f"Selected {recipe_df.shape[0]} from total of {old_len} recipes")

In [ ]:
import ipywidgets as widgets
from IPython.display import display, clear_output, HTML

In [ ]:
unique_cats = (recipe_df.categories.explode().sort_values().unique().tolist())
unique_tags = (recipe_df.tags.explode().unique().tolist())

In [ ]:
# recipe_df.categories.astype(str).str.contains('drink')

In [ ]:
dropdown_cat = widgets.Dropdown(options = unique_cats, description='Category:')
# dropdown_tag1 = widgets.Dropdown(options = unique_tags, description='Tag 1:')
text_tag1 = widgets.Text(value='', description='Tags 1 (or):', continuous_update=False, disabled=False)
text_tag2 = widgets.Text(value='', description='Tags 2 (or):', continuous_update=False, disabled=False)
text_tag3 = widgets.Text(value='', description='Tags 3 (or):', continuous_update=False, disabled=False)

col_sel = ["title", "rating", "time_total", "ingredients", "categories", "tags"]
sort_cols = ["rating"]
top_show = 10

# Create output widget
output = widgets.Output()

def dropdown_eventhandler(change):
    with output:
        clear_output()
        mask = recipe_df['categories'].astype(str).str.contains(dropdown_cat.value)
        for text_tag in [text_tag1, text_tag2, text_tag3]:
            if text_tag.value != '':
                sub_mask = np.zeros(len(mask), dtype=bool)
                all_vals = text_tag.value.split(',')
                for val in all_vals:
                    sub_mask |= recipe_df['tags'].astype(str).str.contains(val.strip())
                mask &= sub_mask
        display(HTML(recipe_df[mask][col_sel].sort_values(by=sort_cols, ascending=False).head(top_show).to_html(index=False)))
    
dropdown_cat.observe(dropdown_eventhandler, names='value')
# Bind function to text box value change
text_tag1.observe(dropdown_eventhandler, names='value')
text_tag2.observe(dropdown_eventhandler, names='value')
text_tag3.observe(dropdown_eventhandler, names='value')

display(dropdown_cat)
display(text_tag1)
display(text_tag2)
display(text_tag3)

display(output)